**Repo** https://github.com/aparna-k/datasci_course_materials/tree/master/assignment1

**Step 1** - Create a twitter app and create an access token. (OAuth1)
http://docs.inboundnow.com/guide/create-twitter-application/

You will now copy four values into the file twitterstream.py. These values are your "Consumer Key (API Key)", your "Consumer Secret (API Secret)", your "Access token" and your "Access token secret". All four should now be visible on the "Keys and Access Tokens" page. (You may see "Consumer Key (API Key)" referred to as either "Consumer key" or "API Key" in some places in the code or on the web; all three are synonyms.) Open twitterstream.py and set the variables corresponding to the api key, api secret, access token, and access secret. You will see code like the below:

In [2]:
api_key = "<Enter api key>" 
api_secret = "<Enter api secret>" 
access_token_key = "<Enter your access token key here>" 
access_token_secret = "<Enter your access token secret here>"

Since I don't want my access token on a public Github repo, I'm creating the following four env variables and using that in the code.

```bash
export TWITTER_API_KEY="xxxx"
export TWITTER_API_SECRET="xxxx"
export TWITTER_ACCESS_TOKEN_KEY="xxx"
export TWITTER_ACCESS_TOKEN_SECRET="xxx"
```

Then in the file `twitterstream.py`

```python
api_key = os.environ.get('TWITTER_API_KEY')
api_secret = os.environ.get('TWITTER_API_SECRET')
access_token_key = os.environ.get('TWITTER_ACCESS_TOKEN_KEY')
access_token_secret = os.environ.get('TWITTER_ACCESS_TOKEN_SECRET')
```